In [13]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.layers import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [14]:
# Download images form github repo

!wget https://github.com/emilwallner/Coloring-greyscale-images/archive/master.tar.gz
!tar xvzf master.tar.gz

X = []
for filename in os.listdir('Coloring-greyscale-images-master/Full-version/Train/'):
    X.append(img_to_array(load_img('Coloring-greyscale-images-master/Full-version/Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

--2021-06-04 12:55:34--  https://github.com/emilwallner/Coloring-greyscale-images/archive/master.tar.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/emilwallner/Coloring-greyscale-images/tar.gz/master [following]
--2021-06-04 12:55:34--  https://codeload.github.com/emilwallner/Coloring-greyscale-images/tar.gz/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘master.tar.gz.9’

master.tar.gz.9         [  <=>               ]   5.76M  22.0MB/s    in 0.3s    

2021-06-04 12:55:34 (22.0 MB/s) - ‘master.tar.gz.9’ saved [6043320]

Coloring-greyscale-images-master/
Coloring-greyscale-images-master/.gitignore
Coloring-

In [15]:
# Create encode, fusion layer and decocer and instantiate model
inception = InceptionResNetV2(weights='imagenet', include_top=True)
embed_input = Input(shape=(1000,))

encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = tf.keras.layers.Concatenate(axis=3)([encoder_output, fusion_output])
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)

decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)


225222656/225209952 [==============================] - 3s 0us/step


In [17]:
# Inception predictions and data generator
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
      i = resize(i, (299, 299, 3), mode='constant')
      grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    embed = inception.predict(grayscaled_rgb_resized)
    return embed

datagen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        rotation_range=40,
        horizontal_flip=True)
        
batch_size = 20

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, embed], Y_batch)


In [18]:
# Compile and Train model
tensorboard = TensorBoard(log_dir="/output")
model.compile(optimizer='adam', loss='mse')
model.fit(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=50, steps_per_epoch=5)


Epoch 1/50
5/5 [==============================] - 15s 637ms/step - loss: 0.1315
Epoch 2/50
5/5 [==============================] - 3s 544ms/step - loss: 0.0079
Epoch 3/50
5/5 [==============================] - 3s 578ms/step - loss: 0.0068
Epoch 4/50
5/5 [==============================] - 3s 577ms/step - loss: 0.0068
Epoch 5/50
5/5 [==============================] - 3s 585ms/step - loss: 0.0057
Epoch 6/50
5/5 [==============================] - 3s 570ms/step - loss: 0.0061
Epoch 7/50
5/5 [==============================] - 3s 580ms/step - loss: 0.0059
Epoch 8/50
5/5 [==============================] - 3s 552ms/step - loss: 0.0058
Epoch 9/50
5/5 [==============================] - 3s 562ms/step - loss: 0.0058
Epoch 10/50
5/5 [==============================] - 3s 580ms/step - loss: 0.0058
Epoch 11/50
5/5 [==============================] - 3s 578ms/step - loss: 0.0055
Epoch 12/50
5/5 [==============================] - 3s 578ms/step - loss: 0.0056
Epoch 13/50
5/5 [==============================]

In [20]:
# Test results and colorize output
color_me = []
for filename in os.listdir('Coloring-greyscale-images-master/Full-version/Test/'):
    color_me.append(img_to_array(load_img('Coloring-greyscale-images-master/Full-version/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = 1.0/255*color_me
color_me = gray2rgb(rgb2gray(color_me))
color_me_embed = create_inception_embedding(color_me)
color_me = rgb2lab(color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

output = model.predict([color_me, color_me_embed])
output = output * 128

for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("Result/img_"+str(i)+".png", lab2rgb(cur))